# Desafio Kinea


## Importando e definindo constantes globais

In [1]:
import requests
import base64 as b64
import json
import pandas as pd
from typing import Dict, List
import csv
from dateutil.relativedelta import relativedelta

In [2]:
import logging
logging.level=logging.INFO

In [3]:
from datetime import date, datetime
from cal import AmbimaCalendar

In [4]:
client_id = "9r2j5fK3DntV"
client_secret = "ufaORoKwRRmw"

## Funções básicas

In [5]:
def dias_uteis(start_date, end_date) -> int:
  """O formato de data é date(AAAA, MM, DD), como date(2022, 2, 4) para 4 de fev de 2022"""
  interval = AmbimaCalendar().get_working_days_delta(start_date, end_date)+1
  return interval

In [6]:
def b64encode(data:str) -> str:
  "Codifica uma string em base64"
  encodedBytes = b64.b64encode(data.encode("utf-8"))
  encodedStr = str(encodedBytes, "utf-8")
  return encodedStr

In [7]:
def check_error(acquired_value, expected_value):
  err = abs((acquired_value-expected_value)/expected_value)*100
  err_str = f'{err:0.3}%'
  return err_str

## Interface com API Ambima

In [8]:
def get_access_token(_id:str, secret:str) -> requests.models.Response:
  """Recebe id e secret de um dado app e retorna o toke de acesso para aquele app"""
  address = "https://api.anbima.com.br/oauth/access-token?"
  credentials = b64encode(f"{_id}:{secret}")
  body = {"grant_type": "client_credentials"}
  headers = {
    "Authorization": f"Basic {credentials}",
    "Content-Type": "application/json"
  }
  logging.debug(f"Auth headers: {headers}")
  res = requests.post(address, headers=headers, json=body)
  logging.info(f"Auth res status: {res.status_code}")
  logging.debug(f"Auth res text: {res.text}")
  return res


In [9]:
def get_data(_id:str, token:str, address:str) -> requests.models.Response:
  headers = {
    "Content-Type": "application/json",
    "access_token": token,
    "client_id": _id}
  logging.debug(f"Auth headers: {headers}")
  res = requests.get(address, headers=headers)
  logging.info(f"Auth res status: {res.status_code}")
  if res.status_code == 401:
    new_token = get_access_token(_id, client_secret)
    get_data(_id, new_token, address)
  logging.debug(f"Auth res text: {res.text}")
  return res

In [10]:
auth_res = get_access_token(client_id, client_secret)
access_token = json.loads(auth_res.content)["access_token"]
logging.info(f"Access Token: {access_token}")

In [11]:
address_base = "https://api-sandbox.anbima.com.br"
address_mercado_secundario_TPF = f"{address_base}/feed/precos-indices/v1/titulos-publicos/mercado-secundario-TPF"
addres_estimativa_selic = f'{address_base}/feed/precos-indices/v1/titulos-publicos/estimativa-selic'
adress_vna = f"{address_base}/feed/precos-indices/v1/titulos-publicos/vna"
adress_ipca_igpm = f"{address_base}/feed/precos-indices/v1/titulos-publicos/projecoes"

data_res = get_data(client_id, access_token, address_mercado_secundario_TPF)
data_list = json.loads(data_res.content)
logging.info(data_list)

In [12]:
data_res = get_data(client_id, access_token, adress_vna)
data_vna = json.loads(data_res.content)
data_vna

[{'data_referencia': '2017-10-20',
  'titulos': [{'tipo_titulo': 'NTN-C',
    'codigo_selic': '770100',
    'index': 0.37,
    'tipo_correcao': 'P',
    'data_validade': '2017-10-13',
    'vna': 3531.425676},
   {'tipo_titulo': 'LFT',
    'codigo_selic': '210100',
    'index': 8.15,
    'tipo_correcao': 'O',
    'data_validade': '2017-10-20',
    'vna': 9162.202647},
   {'tipo_titulo': 'NTN-B',
    'codigo_selic': '760100',
    'index': 0.45,
    'tipo_correcao': 'P',
    'data_validade': '2017-10-17',
    'vna': 3013.085961},
   {'tipo_titulo': 'NTN-B',
    'codigo_selic': '760199',
    'index': 0.45,
    'tipo_correcao': 'P',
    'data_validade': '2017-10-17',
    'vna': 3013.085961}]}]

In [13]:
# [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='210100'][0]

In [14]:
def save_to_csv(filedata:List[Dict], filename:str):
  """Take a list of dicts and save as an .csv with the given name"""
  keys = filedata[0].keys()
  with open('exports\\'+filename, 'w', newline='') as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(filedata)

In [15]:
def clean_mercado_secundario(list_to_clean:List[Dict]):
  keep_keys = ['tipo_titulo','data_referencia','data_vencimento','data_base','taxa_indicativa','pu']
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [16]:
clean_mercado = clean_mercado_secundario(data_list)

## Cálculo de PUs

In [17]:
def get_dias_cupom(start_date, end_date):
  """Retorna uma listagem de cupons entre duas datas, indo da data final até a inicial, de 6 em 6 meses"""
  pgto_date_list = list()
  pgto_date = end_date
  # Gera da lista de datas a partir da data final, em incrementos de 6 meses
  while pgto_date > start_date:
    pgto_date_list.append(pgto_date)
    pgto_date -= relativedelta(months=6)
  # Inverte a ordem da lista para crescente
  pgto_date_list=pgto_date_list[::-1]
  
  return pgto_date_list

LTN

In [36]:
def calcular_ltn(data_venc, taxa, data_ref):
  duration = dias_uteis(data_ref, data_venc)
  pm = (data_venc - data_ref).days
  pu = 1000/((1+taxa)**(duration/252))
  return pu, duration, pm

In [33]:
# calcular_ltn(0.1212, date(2022,3,21), date(2025,1,1))
calcular_ltn(0.0728, date(2017,10,20), date(2018,1,1))

(986.7040298582051, 48, 73)

NTN-F

In [20]:
def calcular_ntn_f(taxa, data_liqui, data_venc):
  lista_datas = get_dias_cupom(data_liqui, data_venc)
  fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
  fluxo_pgto['cupom_semestral'] = [(1.1**(1/2)-1)*1000]*len(lista_datas)
  fluxo_pgto['cupom_semestral'].iloc[-1] = fluxo_pgto['cupom_semestral'].iloc[-1] + 1000
  fluxo_pgto['dias_uteis'] = fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_liqui, x))
  fluxo_pgto['dias_corridos'] = fluxo_pgto['data_pgto'].apply(lambda x: (x - data_liqui).days)
  fluxo_pgto['vp_pgto'] = fluxo_pgto.apply(lambda x: (x['cupom_semestral'])/((1+taxa)**(x['dias_uteis']/252)), axis=1)

  pu = fluxo_pgto['vp_pgto'].sum()
  duration = (fluxo_pgto['vp_pgto']*fluxo_pgto['dias_uteis']).sum()/pu
  pm = (fluxo_pgto['cupom_semestral']*fluxo_pgto['dias_corridos']).sum()/fluxo_pgto['cupom_semestral'].sum()

  return pu, duration, pm


In [ ]:
calcular_ntn_f(0.122, date(2022,3,21), date(2033,1,1))

NTN-B

In [22]:
def calcular_ntnb(taxa, data_liqui, data_venc):
  # d15_base = data_liqui.replace(day=15)
  # d15_proj = d15_base + relativedelta(months=1)
  
  # VNA deveria puxar dados de um banco, porém ainda não existe esse banco e o API tem só uma data em sandbox
  vna = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='760100'][0]

  lista_datas = get_dias_cupom(data_liqui, data_venc)
  fluxo_pgto = pd.DataFrame(lista_datas, columns=['data_pgto'])
  fluxo_pgto['fluxo'] = [(1.06**(1/2)-1)]*len(lista_datas)
  fluxo_pgto['fluxo'].iloc[-1] = fluxo_pgto['fluxo'].iloc[-1] + 1
  fluxo_pgto['dias_uteis'] = fluxo_pgto['data_pgto'].apply(lambda x: dias_uteis(data_liqui, x))
  fluxo_pgto['dias_corridos'] = fluxo_pgto['data_pgto'].apply(lambda x: (x - data_liqui).days)
  fluxo_pgto['vf_fluxo_taxa'] = fluxo_pgto.apply(lambda x: ((1+taxa)**(x['dias_uteis']/252)), axis=1)
  fluxo_pgto['cotacao'] = fluxo_pgto.apply(lambda x: x['fluxo']/x['vf_fluxo_taxa'], axis=1)

  fluxo_pgto['cupom_semestral'] = fluxo_pgto['fluxo']*vna
  fluxo_pgto['vp_pgto'] = fluxo_pgto['cupom_semestral']/fluxo_pgto['vf_fluxo_taxa']

  pu = fluxo_pgto['cotacao'].sum()*vna
  duration = (fluxo_pgto['vp_pgto']*fluxo_pgto['dias_uteis']).sum()/fluxo_pgto['vp_pgto'].sum()
  pm = (fluxo_pgto['cupom_semestral']*fluxo_pgto['dias_corridos']).sum()/fluxo_pgto['cupom_semestral'].sum()
  
  return pu, duration, pm

In [23]:
calcular_ntnb(0.0576, date(2022,3,21), date(2045,5,15))[0]

# print(f'ntn_b_princ_pu: {ntn_b_princ_pu} | ntn_b_princ_prazo_medio: {ntn_b_princ_prazo_medio} | ntn_b_princ_duration: {ntn_b_princ_duration}')
# print(f'ntn_b_princ_pu: {ntn_b_princ_pu}')
# print(f'Erro ntn_b_princ_pu: {check_error(ntn_b_princ_pu, 1059.153)}')

3171.02105752211

LFT

In [41]:
def calcular_lft(data_venc, taxa, data_ref):
  duration = dias_uteis(data_ref, data_venc)
  pm = (data_venc - data_ref).days

  vna_dados = [x['vna'] for x in data_vna[0]['titulos'] if x['codigo_selic']=='210100'][0]
  vna = vna + vna_dados
  cotacao = (1+taxa)**(-duration/252)
  pu = vna*cotacao
  return pu, duration, pm

calcular_lft(date(2022,5,12), 0.001193, date(2022,9,1))

In [39]:
calcular_lft(date(2022,5,12), 0.001193, date(2022,9,1))

IndexError: list index out of range

## Cálculo Geral

In [34]:
for titulo in clean_mercado:
  data_liqui=datetime.strptime(titulo['data_referencia'], "%Y-%m-%d").date()
  data_venc=datetime.strptime(titulo['data_vencimento'], "%Y-%m-%d").date()

  # if titulo['tipo_titulo'] == 'LTN':
  #   pu_calc = calcular_ltn(
  #     taxa_anual=titulo['taxa_indicativa']/100,
  #     data_liqui=data_liqui,
  #     data_venc=data_venc)
  #   titulo['pu_calc']=pu_calc

  # if titulo['tipo_titulo'] == 'NTN-B':
  #   pu_calc = calcular_ntnb(
  #     data_liqui=data_liqui,
  #     data_venc=data_venc,
  #     taxa=titulo['taxa_indicativa']/100
  #     )
  #   titulo['pu_calc']=pu_calc
  
  if titulo['tipo_titulo'] == 'LFT':
      pu_calc = calcular_lft(
        data_liqui=data_liqui,
        data_ref=data_venc,
        taxa=titulo['taxa_indicativa']/100
        )
      titulo['pu_calc']=pu_calc
  


TypeError: calcular_lft() got an unexpected keyword argument 'taxa'

In [30]:
from collections import Counter
Counter([x['tipo_titulo'] for x in clean_mercado])

Counter({'LTN': 11, 'NTN-F': 6, 'LFT': 11, 'NTN-B': 15, 'NTN-C': 2})

In [27]:
t =[x for x in clean_mercado if x['tipo_titulo'] in ['NTN-B']]
print('Real, Calc, Error')
for x in t: print(f"{x['pu']}, {x['pu_calc'][0]}, {check_error(x['pu_calc'][0], x['pu'])}")

Real, Calc, Error
3127.950824, 3127.6391868892265, 0.00996%
3238.274966, 3237.9476706693135, 0.0101%
3227.183797, 3227.136143834944, 0.00148%
3287.138181, 3287.1011284425113, 0.00113%
3258.531943, 3258.0381686381716, 0.0152%
3254.422094, 3253.908583236275, 0.0158%
3321.33068, 3320.8686789625986, 0.0139%
3318.513445, 3317.997025001115, 0.0156%
3329.224966, 3329.078468668097, 0.0044%
3369.672632, 3369.1415223089575, 0.0158%
3456.98885, 3456.5917031636814, 0.0115%
3463.102401, 3462.5870641418596, 0.0149%
3488.327957, 3487.9867039828846, 0.00978%
3487.035343, 3486.529020021143, 0.0145%
3577.298359, 3577.0860027270646, 0.00594%
